In [19]:
# Importing necessary libraries
import sys, os

# Setting the root directory as a string.
root = r'<ROOT LOCATION OF REPORT FOLDER>'

# Combining the root directory with the target directory to create the full path
path = os.path.join(root, "targetdirectory")

# Create empty lists to store the full file paths and file names
fullFilePaths = []
fileNames = []

# Walk through all the subdirectories and files within the root directory
for path, subdirs, files in os.walk(root):
    # Loop through all the files in each subdirectory
    for name in files:
        # Create the full file path by joining the path and file name
        fullFilePaths.append(os.path.join(path, name))

# The final output will be the full file paths of all the files within the specified root directory.

In [20]:
# Import the pandas library to work with dataframes
import pandas as pd

# Specify the filepath of the GRI excel file using a raw string to prevent backslash escaping
griFullFileName = r'<GRI FILE LOCATION>'

# Open the GRI excel file using pd.ExcelFile() and save it to the variable xls
xls = pd.ExcelFile(griFullFileName)

# Create an empty dictionary to store the dataframes
fullGRI = {}

# Loop through each sheet in the Excel file except for the "Overview" sheet
# and read the sheet into a dataframe using pd.read_excel()
# Add each dataframe to the fullGRI dictionary with the sheet name as the key
for sheet_name in xls.sheet_names[1:21]:
    fullGRI[sheet_name] = pd.read_excel(griFullFileName, sheet_name = sheet_name, header=1)

# Read the "Overview" sheet into a dataframe separately and add it to the fullGRI dictionary with the key "Overview"
fullGRI["Overview"] = pd.read_excel(griFullFileName, sheet_name = "Overview")

# Create a new dataframe named total_df by concatenating all dataframes in the fullGRI dictionary except for the "Overview" sheet
# Set ignore_index argument to True to create a new index for the concatenated dataframe
total_df = fullGRI["1999"]
for sheet_name in xls.sheet_names[2:21]:
    total_df = pd.concat([total_df, fullGRI[sheet_name]], ignore_index=True)

# Modify the 'Name' column in total_df by removing all spaces using the str.replace() method
total_df['Name'] = total_df['Name'].str.replace(' ', '')

# The resulting concatenated dataframe total_df contains all GRI data from 1999 to 2018

In [21]:
# Import the regular expressions library to work with patterns
import re

# Create an empty dataframe with two columns named "Organization" and "Year"
df = pd.DataFrame(columns=["Organization", "Year"])

# Loop through each file path in the fullFilePaths list
for filePath in fullFilePaths:
    # Extract the file name without extension from the file path
    fileName = os.path.splitext(os.path.split(filePath)[1])[0]

    # Create a list of two items by splitting the file name using "_" as a separator
    # The first item is the organization name, and the second item is the year
    row = [fileName.split("_")[0], fileName.split("_")[-1]]

    # Add the row to the dataframe at the next available index using df.loc[]
    df.loc[len(df)] = row

# Convert the "Year" column to numeric values using pd.to_numeric()
df['Year'] = pd.to_numeric(df['Year'])

# The resulting dataframe df contains information about the PDF files, including the organization name and year

In [66]:
# INPUT: GRI DATAFRAME and PDF FILES DATAFRAME
import numpy as np

# merge the two dataframes based on 'Organization' and 'Year' columns
almost_df = pd.merge(df, total_df,  how='left', left_on=['Organization','Year'], right_on = ['Name','Publication Year'])

# drop the 'Name' and 'Publication Year' columns from the merged dataframe
final_df = almost_df.drop(['Name','Publication Year'], axis=1)

# drop duplicate rows based on 'Organization' and 'Year' columns
final_df = final_df.drop_duplicates(subset=['Organization', 'Year']).reset_index(drop=True)

# add a new column called "WordCount" with NaN values
final_df["WordCount"] = np.nan

# OUTPUT: ONE FULL MERGED DATAFRAME WITH ALL THE PDF FILES AND GRI INFORMATION

In [6]:
# Import the stopwords from the NLTK corpus
from nltk.corpus import stopwords

# Create an empty set to store all stop words
all_stop_words = set(stopwords.words('english'))

# Define a list of additional languages for which to include stopwords
languages = ["spanish", "chinese", "portuguese", "greek", "russian", 
             "italian", "finnish", "german", "indonesian", "norwegian", 
             "swedish", "french", "turkish", "hungarian", "romanian"]

# Loop through each language and add the corresponding stopwords to the set
for language in languages:
    stop_words = set(stopwords.words(language))
    all_stop_words = all_stop_words.union(stop_words)

# The purpose of this code is to create a big list of stopwords for multiple languages. 
# The first step is to initialize an empty set with the stop words for the English language, as provided by the nltk.corpus module.
# Next, a list of additional languages for which to include stopwords is defined. 
# A loop is then used to iterate over each language in the list, and for each language, the corresponding set of stopwords is obtained using the stopwords.words() function and stored in the 'stop_words' variable.
# The union of the 'stop_words' set and the 'all_stop_words' set is then taken and stored in the 'all_stop_words' set, to create a bigger set of stopwords for multiple languages. 
# The final output is a big list of all the needed stopwords.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
import re

# Loop through each file path in the list
index = 0
for filePath in fullFilePaths:
    # Print the current index after every 10 files are processed
    if index % 10 == 0:
        print(index)
    
    try:
        # Create a pdf reader object and an empty text variable
        reader = PdfReader(filePath)
        text = ""

        # Extract the text from each page and append it to the text variable
        for page_number in range(len(reader.pages)):
            page = reader.pages[page_number]
            page_text = page.extract_text().lower()
            text += page_text

        # Remove non-alphabetic characters and extra whitespace
        clean_text = re.sub(r'[^a-zA-Z\s]', '', text).replace('\n', '')
        clean_text = re.sub(r'\s{2,}', ' ', clean_text).lower()

        # Tokenize the text into individual words
        tokens = word_tokenize(clean_text)

        # Remove single-letter words and stopwords
        clean_tokens = [i for i in tokens if len(i) > 1]
        filtered_tokens = [word for word in clean_tokens if not word in all_stop_words]

        # Lemmatize the words to their root form
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

        # Count the frequency of each word and store the 5000 most common words
        counts = FreqDist(lemmatized_tokens)
        token_counts = counts.most_common(5000)
    except:
        print("An exception occurred")
        token_counts = "An Error occured and no information was gathered"
    
    # Add the token counts to the final dataframe
    final_df.at[index, "WordCount"] = token_counts
    
    index += 1

In [ ]:
# Define the file location to which the final dataframe will be exported
file_location = r'<EXPORT FILE LOCATION>'

# Export the final dataframe to the defined file location as a CSV file, without the index column
final_df.to_csv(file_location, index=False)